In [1]:
# Phase 3.5: Neural Baseline Optimization
# Advanced LSTM with Enhanced Features and Preprocessing

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add project source to path
sys.path.insert(0, '../src')

# Import our existing infrastructure
from data.evaluation_cases import EvaluationCaseManager
from preprocessing.advanced_features import Phase35DataProcessor
from models.optimized_lstm import Phase35LSTMTrainer

# PyTorch imports
import torch
import torch.nn as nn

print("=" * 60)
print("🚀 PHASE 3.5: NEURAL BASELINE OPTIMIZATION")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

print("\n🎯 OPTIMIZATION OBJECTIVES:")
print("   Current Neural RMSLE: 0.5466 (-15% vs traditional)")
print("   Target Neural RMSLE: < 0.38 (+20% vs traditional)")
print("   Success Criteria: 7-10 cases beat traditional baseline")

🚀 PHASE 3.5: NEURAL BASELINE OPTIMIZATION
PyTorch version: 2.2.2
CUDA available: False

🎯 OPTIMIZATION OBJECTIVES:
   Current Neural RMSLE: 0.5466 (-15% vs traditional)
   Target Neural RMSLE: < 0.38 (+20% vs traditional)
   Success Criteria: 7-10 cases beat traditional baseline


In [2]:
# ============================================================================
# SECTION 1: LOAD INFRASTRUCTURE AND DATA
# ============================================================================

print("\n📋 Section 1: Infrastructure Setup")
print("-" * 40)

# Load evaluation cases (same 10 cases from Phase 2)
case_manager = EvaluationCaseManager()
evaluation_cases = case_manager.get_cases_list()

print(f"✅ Loaded {len(evaluation_cases)} evaluation cases")
print(f"✅ Using established train/test split: 2017-07-01")

# Display case summary
print("\n📊 Evaluation Cases Summary:")
for i, case in enumerate(evaluation_cases, 1):
    store = case['store_nbr']
    family = case['family']
    quality = case.get('selection_metrics', {}).get('quality_score', 0)
    print(f"  {i:2d}. Store {store:2d} - {family:30s} (Quality: {quality:.1f})")


📋 Section 1: Infrastructure Setup
----------------------------------------
✅ Loaded 10 evaluation cases
✅ Using established train/test split: 2017-07-01

📊 Evaluation Cases Summary:
   1. Store 49 - PET SUPPLIES                   (Quality: 98.3)
   2. Store  8 - PET SUPPLIES                   (Quality: 98.1)
   3. Store 44 - SCHOOL AND OFFICE SUPPLIES     (Quality: 99.2)
   4. Store 45 - SCHOOL AND OFFICE SUPPLIES     (Quality: 99.0)
   5. Store 39 - MEATS                          (Quality: 98.6)
   6. Store 53 - PRODUCE                        (Quality: 98.5)
   7. Store 26 - FROZEN FOODS                   (Quality: 98.9)
   8. Store 46 - SCHOOL AND OFFICE SUPPLIES     (Quality: 98.8)
   9. Store 47 - SCHOOL AND OFFICE SUPPLIES     (Quality: 98.8)
  10. Store 48 - SCHOOL AND OFFICE SUPPLIES     (Quality: 98.6)


In [7]:
# ============================================================================
# SECTION 2: LOAD PHASE 2 & 3 BASELINE RESULTS FOR COMPARISON
# ============================================================================

print("\n📈 Section 2: Loading Baseline Results for Comparison")
print("-" * 40)

try:
    # Load traditional baseline results from Phase 2
    import json
    with open('../results/traditional_baseline_results.json', 'r') as f:
        traditional_results = json.load(f)
    
    traditional_summary = traditional_results['summary_statistics']
    print("✅ Traditional Baseline Results Loaded:")
    
    # Best traditional baseline
    best_traditional = min(traditional_summary.items(), key=lambda x: x[1]['mean_rmsle'])
    best_model_name, best_stats = best_traditional
    traditional_baseline_rmsle = best_stats['mean_rmsle']
    
    print(f"\n🎯 Best Traditional Model: {best_model_name}")
    print(f"   Baseline RMSLE: {traditional_baseline_rmsle:.4f}")
    
except FileNotFoundError:
    print("⚠️ Traditional results not found - using specified baseline")
    traditional_baseline_rmsle = 0.4755

try:
    # Load Phase 3 neural baseline results
    with open('../results/neural_baseline_results.json', 'r') as f:
        neural_results = json.load(f)
    
    neural_summary = neural_results['summary_statistics']
    
    # FIXED: Find best neural model by comparing mean_rmsle values
    best_neural_model = None
    best_neural_rmsle = float('inf')
    
    for model_name, stats in neural_summary.items():
        if stats['mean_rmsle'] < best_neural_rmsle:
            best_neural_rmsle = stats['mean_rmsle']
            best_neural_model = model_name
    
    current_neural_rmsle = best_neural_rmsle
    
    print(f"✅ Phase 3 Neural Results Loaded:")
    print(f"   Best Neural Model: {best_neural_model}")
    print(f"   Current Neural RMSLE: {current_neural_rmsle:.4f}")
    
except FileNotFoundError:
    print("⚠️ Neural results not found - using specified baseline")
    current_neural_rmsle = 0.5466

# Performance targets
target_rmsle = traditional_baseline_rmsle * 0.8  # 20% improvement
stretch_rmsle = traditional_baseline_rmsle * 0.75  # 25% improvement

print(f"\n🎯 PHASE 3.5 PERFORMANCE TARGETS:")
print(f"   Traditional Baseline: {traditional_baseline_rmsle:.4f}")
print(f"   Current Neural: {current_neural_rmsle:.4f} ({((current_neural_rmsle - traditional_baseline_rmsle) / traditional_baseline_rmsle * 100):+.1f}%)")
print(f"   Target Neural: {target_rmsle:.4f} (20% improvement)")
print(f"   Stretch Target: {stretch_rmsle:.4f} (25% improvement)")


📈 Section 2: Loading Baseline Results for Comparison
----------------------------------------
⚠️ Traditional results not found - using specified baseline
✅ Phase 3 Neural Results Loaded:
   Best Neural Model: lstm_with_features
   Current Neural RMSLE: 0.5466

🎯 PHASE 3.5 PERFORMANCE TARGETS:
   Traditional Baseline: 0.4755
   Current Neural: 0.5466 (+15.0%)
   Target Neural: 0.3804 (20% improvement)
   Stretch Target: 0.3566 (25% improvement)


In [11]:
# ============================================================================
# SECTION 3: INITIALIZE PHASE 3.5 OPTIMIZATION COMPONENTS
# ============================================================================

print("\n🔧 Section 3: Initializing Phase 3.5 Components")
print("-" * 40)

# IMPORTANT: Reload the module to get latest fixes
import importlib
import preprocessing.advanced_features
importlib.reload(preprocessing.advanced_features)
from preprocessing.advanced_features import Phase35DataProcessor

print("🔄 Reloaded Phase35DataProcessor with latest fixes")

# Initialize enhanced data processor
processor = Phase35DataProcessor()
print("✅ Phase 3.5 Data Processor initialized")

# Initialize optimized LSTM trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = Phase35LSTMTrainer(case_manager, device=device)
print(f"✅ Phase 3.5 LSTM Trainer initialized (device: {device})")

# Results storage
optimization_results = []
processing_summary = {
    'attempted': 0,
    'successful_preprocessing': 0,
    'successful_training': 0,
    'successful_evaluation': 0,
    'failed_cases': []
}

print("✅ Results tracking initialized")


🔧 Section 3: Initializing Phase 3.5 Components
----------------------------------------
🔄 Reloaded Phase35DataProcessor with latest fixes
🔧 Phase35DataProcessor initialized
✅ Phase 3.5 Data Processor initialized
🚀 Phase35LSTMTrainer initialized
   Device: cpu
   EvaluationCaseManager: ✅ Connected
✅ Phase 3.5 LSTM Trainer initialized (device: cpu)
✅ Results tracking initialized


In [12]:
# ============================================================================
# SECTION 4: PHASE 3.5 OPTIMIZATION LOOP
# ============================================================================

print("\n🚀 Section 4: Phase 3.5 Optimization Loop")
print("=" * 60)
print(f"Target: Beat traditional baseline of {traditional_baseline_rmsle:.4f} RMSLE")
print(f"Goal: Achieve < {target_rmsle:.4f} RMSLE (20% improvement)")
print("=" * 60)

for i, case in enumerate(evaluation_cases, 1):
    store_id = case['store_nbr']
    family = case['family']
    quality_score = case.get('selection_metrics', {}).get('quality_score', 0)
    
    print(f"\n🔥 CASE {i}/10: Store {store_id}, {family} (Quality: {quality_score:.1f})")
    print("-" * 50)
    
    processing_summary['attempted'] += 1
    
    try:
        # Step 1: Advanced preprocessing
        print("📊 Step 1: Advanced preprocessing...")
        train_processed, test_processed, feature_columns, scaler = processor.process_evaluation_case(
            case_manager, store_id, family
        )
        
        # Verify sufficient data
        if len(train_processed) < 100 or len(test_processed) < 15:
            print(f"❌ Insufficient data: train={len(train_processed)}, test={len(test_processed)}")
            processing_summary['failed_cases'].append({
                'store_id': store_id, 'family': family, 'reason': 'insufficient_data'
            })
            continue
        
        processing_summary['successful_preprocessing'] += 1
        
        # Step 2: Train optimized model
        print("🚀 Step 2: Training optimized LSTM...")
        model = trainer.train_model(
            train_processed, feature_columns, store_id, family,
            epochs=100, batch_size=32, learning_rate=0.001
        )
        
        if model is None:
            print(f"❌ Model training failed")
            processing_summary['failed_cases'].append({
                'store_id': store_id, 'family': family, 'reason': 'training_failed'
            })
            continue
        
        processing_summary['successful_training'] += 1
        
        # Step 3: Evaluate performance
        print("📈 Step 3: Evaluating performance...")
        evaluation = trainer.evaluate_model(model, test_processed, feature_columns, scaler)
        
        rmsle = evaluation['rmsle']
        
        if rmsle == float('inf'):
            print(f"❌ Evaluation failed")
            processing_summary['failed_cases'].append({
                'store_id': store_id, 'family': family, 'reason': 'evaluation_failed'
            })
            continue
        
        processing_summary['successful_evaluation'] += 1
        
        # Calculate improvements
        improvement_vs_traditional = (traditional_baseline_rmsle - rmsle) / traditional_baseline_rmsle * 100
        improvement_vs_neural = (current_neural_rmsle - rmsle) / current_neural_rmsle * 100
        
        # Store results
        optimization_results.append({
            'store_id': store_id,
            'family': family,
            'quality_score': quality_score,
            'rmsle': rmsle,
            'improvement_vs_traditional': improvement_vs_traditional,
            'improvement_vs_neural': improvement_vs_neural,
            'beats_traditional': rmsle < traditional_baseline_rmsle,
            'beats_target': rmsle < target_rmsle,
            'beats_stretch': rmsle < stretch_rmsle,
            'n_features': len(feature_columns),
            'n_predictions': evaluation['n_predictions']
        })
        
        # Progress report
        if rmsle < stretch_rmsle:
            status = "🌟 STRETCH!"
        elif rmsle < target_rmsle:
            status = "🎯 TARGET!"
        elif rmsle < traditional_baseline_rmsle:
            status = "✅ GOOD"
        else:
            status = "⚠️ NEEDS WORK"
        
        print(f"📊 RESULT: RMSLE = {rmsle:.4f}")
        print(f"   vs Traditional: {improvement_vs_traditional:+.1f}%")
        print(f"   vs Neural: {improvement_vs_neural:+.1f}%")
        print(f"   Status: {status}")
        
    except Exception as e:
        print(f"❌ Error processing case: {e}")
        processing_summary['failed_cases'].append({
            'store_id': store_id, 'family': family, 'reason': f'exception: {str(e)}'
        })
        continue

print(f"\n🏁 PHASE 3.5 OPTIMIZATION COMPLETE!")


🚀 Section 4: Phase 3.5 Optimization Loop
Target: Beat traditional baseline of 0.4755 RMSLE
Goal: Achieve < 0.3804 RMSLE (20% improvement)

🔥 CASE 1/10: Store 49, PET SUPPLIES (Quality: 98.3)
--------------------------------------------------
📊 Step 1: Advanced preprocessing...

🔧 PROCESSING CASE: Store 49, PET SUPPLIES
   ✅ Loaded sales data from: ../data/raw/train.csv
   Raw train data: 1638 records
   Raw test data: 46 records
🔄 Applying log1p transformation...
   Sales range: 0.00 to 59.00
   Log1p range: 0.0000 to 4.0943
🔄 Applying log1p transformation...
   Sales range: 7.00 to 45.00
   Log1p range: 2.0794 to 3.8286
🔄 Creating cyclical temporal features...
   ✅ Created 6 cyclical features
🔄 Creating cyclical temporal features...
   ✅ Created 6 cyclical features
🔄 Creating enhanced features for Store 49, PET SUPPLIES...
   ✅ Created 43 enhanced features
🔄 Creating enhanced features for Store 49, PET SUPPLIES...
   ✅ Created 43 enhanced features
📊 Total features identified: 48
🔄 Ap

In [ ]:
# ============================================================================
# QUICK TEST: Verify the preprocessing fix is working
# ============================================================================

print("🧪 Quick Test: Preprocessing Fix Verification")
print("-" * 50)

# Test with one case to verify the fix
test_case = evaluation_cases[0]  # First case: Store 49, PET SUPPLIES
store_id = test_case['store_nbr'] 
family = test_case['family']

print(f"Testing Store {store_id}, {family}...")

try:
    result = processor.process_evaluation_case(case_manager, store_id, family)
    
    if result is not None:
        train_data, test_data, features, scaler = result
        print(f"✅ SUCCESS! Preprocessing working correctly")
        print(f"   Train shape: {train_data.shape}")
        print(f"   Test shape: {test_data.shape}")
        print(f"   Features: {len(features)}")
        print("\n🚀 Ready to run the optimization loop!")
    else:
        print("❌ Preprocessing returned None")
        
except Exception as e:
    print(f"❌ Still getting error: {e}")
    print("💡 Try restarting the kernel (Kernel → Restart) and re-run all cells")

print("\n" + "=" * 50)